<a href="https://colab.research.google.com/github/adikal25/Automated-Clinical-Documentation-from-Patient-Encounters/blob/main/patient_doctor_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from openai import OpenAI
import os
from google.colab import userdata
import base64

api_key = userdata.get("OR_A_TOKEN")

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

patient_audio_path = "/content/100301_1o00.wav"
doctor_audio_path = "/content/100301_2o00.wav"

# Read binary audio and encode in Base64
with open(patient_audio_path, "rb") as f:
    patient_audio_bytes = f.read()
    patient_audio_base64 = base64.b64encode(patient_audio_bytes).decode('utf-8')

with open(doctor_audio_path, "rb") as f:
    doctor_audio_bytes = f.read()
    doctor_audio_base64 = base64.b64encode(doctor_audio_bytes).decode('utf-8')


PROMPT = """
You are a medical note generator.

Audio Speakers:
- First audio file = Patient
- Second audio file = Doctor/Clinician

Generate a detailed and structured SOAP note based on the provided medical conversation.

Subjective (S):
Summarize patient-reported symptoms, chief complaints, medical history, and subjective experiences. Only include patient perspective.

Objective (O):
Include measurable and observable clinical findings mentioned by the doctor (vitals, exam findings, labs, imaging). If not stated, write “Not documented in transcript.”

Assessment (A):
Provide the clinical diagnosis or differential based solely on the transcript.

Plan (P):
Outline treatments, medications, tests ordered, referrals, counseling, and follow-up instructions mentioned.

Use concise, professional clinical language. Do not invent information. De-identify the note and use only transcript content.

"""

completion = client.chat.completions.create(
    model="openai/gpt-4o-audio-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": PROMPT},
                {"type": "input_audio", "input_audio": {"data": patient_audio_base64, "format": "wav"}},
                {"type": "input_audio", "input_audio": {"data": doctor_audio_base64, "format": "wav"}}
            ]
        }
    ]
)

print(completion.choices[0].message.content)

**SOAP Note**

**Subjective (S):**
- The patient reports experiencing asthma attacks and "pins and needles" sensations in the chest, arms, and hands.
- The patient states that their family has been ill with chest congestion, and all have been coughing a lot.
- The patient has had two heart stents placed within the last 11 months.
- The patient mentions not having had asthma attacks for a few years.
- The patient is unsure whether to attempt to stay comfortable and endure the sensations when coughing, or to seek medical attention.

**Objective (O):**
Not documented in transcript.

**Assessment (A):**
- Possible nerve irritation post-stent placement.
- Asthma exacerbation may be contributing to symptoms.
- Chest congestion and cough, likely due to viral or infectious etiology as the family is similarly affected.

**Plan (P):**
- Reassurance was provided that some nerve disturbance could occur following stent placement.
- No urgent concerns were expressed by the clinician during the discu